In [28]:
#Importamos las librerias que se ocuaparan 
import pandas as pd
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk

In [29]:
#Ruta del archivo con el que trabajaremos
df = pd.read_parquet(r"C:\Users\Elias\Documents\Primer Proyecto Individual\Datasets limpios\reviews_clean.parquet")

# Copia del DataFrame original para mantener los datos intactos
df_sentimiento = df.copy()
#Mostramos el DataFrame
df_sentimiento.head()

,user_id,user_url,posted,item_id,helpful,recommend,sentiment_scores,review_clean,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted November 5, 2011.",1250,No ratings yet,True,"{'compound': 0.8481, 'neg': 0.077, 'neu': 0.68...",Simple yet with great replayability. In my opi...,2
1,js41637,http://steamcommunity.com/id/js41637,"Posted July 15, 2011.",22200,No ratings yet,True,"{'compound': 0.2263, 'neg': 0.0, 'neu': 0.678,...",It's unique and worth a playthrough.,2
2,evcentric,http://steamcommunity.com/id/evcentric,"Posted April 21, 2011.",43110,No ratings yet,True,"{'compound': 0.9117, 'neg': 0.0, 'neu': 0.704,...",Great atmosphere. The gunplay can be a bit chu...,2
3,doctr,http://steamcommunity.com/id/doctr,"Posted June 24, 2014.",251610,15 of 20 people (75%) found this review helpful,True,"{'compound': 0.9566, 'neg': 0.067, 'neu': 0.71...",I know what you think when you see this title ...,2
4,maplemage,http://steamcommunity.com/id/maplemage,"Posted September 8, 2013.",227300,0 of 1 people (0%) found this review helpful,True,"{'compound': 0.9708, 'neg': 0.042, 'neu': 0.76...",For a simple (it's actually not all that simpl...,2


In [30]:
#ver cuantos valores nulos tengo en la columna review
df_sentimiento['review_clean'].isnull().sum()
#quitar los valores nulos en la columna review
df_sentimiento = df_sentimiento.dropna(subset=['review_clean'])

In [31]:
#instanciar el analizador de sentimiento
sid = SentimentIntensityAnalyzer()

#funcion para calcular el sentimiento
def sentiment_scores(review):
    ss = sid.polarity_scores(review)
    return ss

#aplicar la función a cada reseña
df_sentimiento['sentiment_scores'] = df_sentimiento['review_clean'].apply(sentiment_scores)



In [32]:
#Remplazamos los valores en formato de emoticon en vacio
df_sentimiento['review_clean'] = df['review_clean'].str.replace('♥','', regex=False)


#Mostramos el Dataframe
df_sentimiento.head()

,user_id,user_url,posted,item_id,helpful,recommend,sentiment_scores,review_clean,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted November 5, 2011.",1250,No ratings yet,True,"{'neg': 0.077, 'neu': 0.681, 'pos': 0.242, 'co...",Simple yet with great replayability. In my opi...,2
1,js41637,http://steamcommunity.com/id/js41637,"Posted July 15, 2011.",22200,No ratings yet,True,"{'neg': 0.0, 'neu': 0.678, 'pos': 0.322, 'comp...",It's unique and worth a playthrough.,2
2,evcentric,http://steamcommunity.com/id/evcentric,"Posted April 21, 2011.",43110,No ratings yet,True,"{'neg': 0.0, 'neu': 0.704, 'pos': 0.296, 'comp...",Great atmosphere. The gunplay can be a bit chu...,2
3,doctr,http://steamcommunity.com/id/doctr,"Posted June 24, 2014.",251610,15 of 20 people (75%) found this review helpful,True,"{'neg': 0.067, 'neu': 0.719, 'pos': 0.214, 'co...",I know what you think when you see this title ...,2
4,maplemage,http://steamcommunity.com/id/maplemage,"Posted September 8, 2013.",227300,0 of 1 people (0%) found this review helpful,True,"{'neg': 0.042, 'neu': 0.766, 'pos': 0.192, 'co...",For a simple (it's actually not all that simpl...,2


In [33]:

sid = SentimentIntensityAnalyzer()
# Función para obtener el puntaje de sentimiento
def sentiment_scores(review):
    return sid.polarity_scores(review)

# Aplicar la función y crear una nueva columna con los puntajes
df_sentimiento['sentiment_scores'] = df_sentimiento['review_clean'].apply(sentiment_scores)

# Función para clasificar el sentimiento en la escala 0, 1, 2
def classify_sentiment(sentiment, review):
    if pd.isna(review) or review.strip() == "":  # Verificar si la reseña está vacía o es NaN
        return 1  # Sentimiento neutral
    compound = sentiment['compound']
    if compound <= -0.05:  # Umbral para sentimiento negativo
        return 0
    elif compound >= 0.05:  # Umbral para sentimiento positivo
        return 2
    else:  # Sentimiento neutral
        return 1

# Aplicar la clasificación de sentimientos
df_sentimiento['sentiment_analysis'] = df_sentimiento.apply(lambda x: classify_sentiment(x['sentiment_scores'], x['review_clean']), axis=1)

# Mostrar los resultados
df_sentimiento.head()

,user_id,user_url,posted,item_id,helpful,recommend,sentiment_scores,review_clean,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted November 5, 2011.",1250,No ratings yet,True,"{'neg': 0.077, 'neu': 0.681, 'pos': 0.242, 'co...",Simple yet with great replayability. In my opi...,2
1,js41637,http://steamcommunity.com/id/js41637,"Posted July 15, 2011.",22200,No ratings yet,True,"{'neg': 0.0, 'neu': 0.678, 'pos': 0.322, 'comp...",It's unique and worth a playthrough.,2
2,evcentric,http://steamcommunity.com/id/evcentric,"Posted April 21, 2011.",43110,No ratings yet,True,"{'neg': 0.0, 'neu': 0.704, 'pos': 0.296, 'comp...",Great atmosphere. The gunplay can be a bit chu...,2
3,doctr,http://steamcommunity.com/id/doctr,"Posted June 24, 2014.",251610,15 of 20 people (75%) found this review helpful,True,"{'neg': 0.067, 'neu': 0.719, 'pos': 0.214, 'co...",I know what you think when you see this title ...,2
4,maplemage,http://steamcommunity.com/id/maplemage,"Posted September 8, 2013.",227300,0 of 1 people (0%) found this review helpful,True,"{'neg': 0.042, 'neu': 0.766, 'pos': 0.192, 'co...",For a simple (it's actually not all that simpl...,2


In [34]:
#Convertimos el Dataframe en un dataset de tipo parquet
df_sentimiento.to_parquet(r"C:\Users\Elias\Documents\Primer Proyecto Individual\Datasets limpios\reviews_sentiment.parquet")